# Euler 7

https://projecteuler.net/problem=7

> By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.

> What is the 10 001st prime number?

I guess the fastest solution is by using a sieve.

*    Find a number which, for sure, is greater then the nth prime
*https://en.wikipedia.org/wiki/Prime-counting_function#Inequalities*

In [1]:
function get_upper_bound(nth_prime)
    ub::Integer = ceil(nth_prime * log(nth_prime * log(nth_prime)))
    max(ub, 7)
end

get_upper_bound (generic function with 1 method)

Julia has a dynamic type system, but we can add [type annotation][type_doc] to variables, like *ub*, to 
generate more efficient code.

Moreover, `get_upper_bound` is a *generic function*. We can write several implementations of it, by adding different
type specialization of its arguments ([*multimethods*][methods]). Then, Julia chooses the concrete implementation to run by implementing *multiple dispatch*.

In [11]:
target_nth_prime = 10001
ub = get_upper_bound(target_nth_prime)

114320

*    Build an array of booleans $v$, s.t. $v[i]$ is false if $i$ is prime.

Julia has a very convenient bit array for this purpose. **Julia arrays are indexed starting from 1, not 0.**

In [3]:
v = trues(ub)
v[0]

LoadError: BoundsError()
while loading In[3], in expression starting on line 2

*    run the sieve and get the nth prime

First of all I try the [Sieve of Eratosthenes][eratosthenes].

In [4]:
function eratosthenes_sieve(target_nth)
    n = get_upper_bound(target_nth)
    root_n::Integer = floor(sqrt(n))
    sieve = trues(n)
    sieve[1] = false
    
    for i = 2:root_n
        if sieve[i]
            # this is faster than v[i^2:i:n] = false
            for j=i^2:i:n
                sieve[j] = false
            end
            target_nth -= 1
            if 0 == target_nth
                return i
            end
        end
    end
    
    # I keep going past sqrt(n) until the target_nth
    for i = root_n+1:n
        if sieve[i]
            target_nth -= 1
            if 0 == target_nth
                return i
            end
        end
    end
end

@time eratosthenes_sieve(target_nth_prime)

elapsed time: 0.014508443 seconds (430144 bytes allocated)


104743

We measure elapsed time by using the time macro.

The [sieve of Sundaram][sundaram] is slightly more complex. Compared to Eratosthenes's, it skips even numbers.


In [5]:
function sundaram_sieve(target_nth)
    ub = get_upper_bound(target_nth)
    sieve = trues(ub)
    n = div(ub, 2)
    target_nth -= 1 # it skips 2
    for i = 1:n
        step = i * 2 + 1
        for j=i+i*step:step:n
            sieve[j] = false
        end
        if sieve[i]
            target_nth -= 1
            if 0 == target_nth
                return i*2+1
            end
        end
    end
end


@time sundaram_sieve(target_nth_prime)

elapsed time: 0.010242764 seconds (293692 bytes allocated)


104743

In [6]:
function get_bitmask(n, idx)
    f = falses(n)
    for i in idx
        f[i] = true
    end
    f
end

# http://www.codeproject.com/Articles/490085/Eratosthenes-Sundaram-Atkins-Sieve-Implementation
function atkin_sieve(target_nth)
    
    if target_nth < 4
        return eratosthenes_sieve(target_nth)
    end
    
    ub = get_upper_bound(target_nth)
    root_ub::Integer = floor(sqrt(ub))
    target_nth -= 3
    is_prime = get_bitmask(ub, [2, 3, 5])
    
    step31_filter = get_bitmask(60, [1, 13, 17, 29, 37, 41, 49, 53])
    step32_filter = get_bitmask(60, [7, 19, 31, 43])
    step33_filter = get_bitmask(60, [11, 23, 47, 59])
    
    x, x_square, x_step = 1, 1, 3
    
    while x <= root_ub
    
        y, y_square, y_step = 1, 1, 3
        while y <= root_ub
            # 4x^2 + y^2
            n = (x_square << 2) + y_square
            n_mod_60 = n % 60
            if n <= ub && n_mod_60 > 0 && step31_filter[n_mod_60]
                is_prime[n] = !is_prime[n]
            end
            
            # 3x^2 + y^2
            n -= x_square
            n_mod_60 = n % 60
            if n <= ub && n_mod_60 > 0 && step32_filter[n_mod_60]
                is_prime[n] = !is_prime[n]
            end
            
            if x > y
                n -= (y_square << 1)
                n_mod_60 = n % 60
                if n <= ub && n_mod_60 > 0 && step33_filter[n_mod_60]
                    is_prime[n] = !is_prime[n]
                end
            end
            
            y += 1
            y_square += y_step
            y_step += 2
        end
        x += 1
        x_square += x_step
        x_step += 2
    end
    
    for n = 7:root_ub
        if is_prime[n]
            target_nth -= 1
            if 0 == target_nth
                return n
            end
            
            n_squared = n^2
            for c = n_squared:n_squared:ub
                is_prime[c] = false
            end
        end
    end
    
    for n = root_ub+1:ub
        if is_prime[n]
            target_nth -= 1
            if 0 == target_nth
                return n
            end
        end
    end
end

@time atkin_sieve(target_nth_prime)

elapsed time: 0.032403499 seconds (1497144 bytes allocated)


104743

In [7]:
function bench_sieve(nth_prime, generator, runs=5)
    times = zeros(runs)
    # run the first time just to ensure it is compiled
    generator(nth_prime)
    for i = 1:runs
        times[i] = @elapsed generator(nth_prime)
    end
    mean(times)
end


bench_sieve (generic function with 2 methods)

In [8]:
using DataFrames

sieves = [:Erathostenes => eratosthenes_sieve,
          :Sundaram => sundaram_sieve,
          :Atkin => atkin_sieve]

df = DataFrame(nths = map(round, logspace(1, 7, 7)))
for (name, sieve) in sieves
    df[name] = map(nth -> bench_sieve(nth, sieve), df[:nths])
end


In [9]:
# stack all columns but nths
melt(df, :nths)

,variable,value,nths
1,Erathostenes,9.288e-7,10.0
2,Erathostenes,9.527e-6,100.0
3,Erathostenes,0.0001451148,1000.0
4,Erathostenes,0.0020518454,10000.0
5,Erathostenes,0.026853671000000003,100000.0
6,Erathostenes,0.337851787,1.0e6
7,Erathostenes,4.207650377,1.0e7
8,Atkin,1.465e-6,10.0
9,Atkin,1.03086e-5,100.0
10,Atkin,0.000142602,1000.0


In [10]:
# to install Gadfly
# Pkg.add("Gadfly")
using Gadfly

set_default_plot_size(24cm, 16cm)
p = plot(melt(df, :nths), x = :nths,
         y = :value, color=:variable,
         Guide.xlabel("nth prime"),
         Guide.ylabel("elapsed time [s]"),
         Geom.point,
         Geom.line)
display(p)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 nth prime 
 
 
 -1.5×10⁷ 
 -1.0×10⁷ 
 -5.0×10⁶ 
 0 
 5.0×10⁶ 
 1.0×10⁷ 
 1.5×10⁷ 
 2.0×10⁷ 
 2.5×10⁷ 
 -1.00×10⁷ 
 -9.50×10⁶ 
 -9.00×10⁶ 
 -8.50×10⁶ 
 -8.00×10⁶ 
 -7.50×10⁶ 
 -7.00×10⁶ 
 -6.50×10⁶ 
 -6.00×10⁶ 
 -5.50×10⁶ 
 -5.00×10⁶ 
 -4.50×10⁶ 
 -4.00×10⁶ 
 -3.50×10⁶ 
 -3.00×10⁶ 
 -2.50×10⁶ 
 -2.00×10⁶ 
 -1.50×10⁶ 
 -1.00×10⁶ 
 -5.00×10⁵ 
 0 
 5.00×10⁵ 
 1.00×10⁶ 
 1.50×10⁶ 
 2.00×10⁶ 
 2.50×10⁶ 
 3.00×10⁶ 
 3.50×10⁶ 
 4.00×10⁶ 
 4.50×10⁶ 
 5.00×10⁶ 
 5.50×10⁶ 
 6.00×10⁶ 
 6.50×10⁶ 
 7.00×10⁶ 
 7.50×10⁶ 
 8.00×10⁶ 
 8.50×10⁶ 
 9.00×10⁶ 
 9.50×10⁶ 
 1.00×10⁷ 
 1.05×10⁷ 
 1.10×10⁷ 
 1.15×10⁷ 
 1.20×10⁷ 
 1.25×10⁷ 
 1.30×10⁷ 
 1.35×10⁷ 
 1.40×10⁷ 
 1.45×10⁷ 
 1.50×10⁷ 
 1.55×10⁷ 
 1.60×10⁷ 
 1.65×10⁷ 
 1.70×10⁷ 
 1.75×10⁷ 
 1.80×10⁷ 
 1.85×10⁷ 
 1.90×10⁷ 
 1.95×10⁷ 
 2.00×10⁷ 
 -1×10⁷ 
 0 
 1×10⁷ 
 2×10⁷ 
 -1.0×10⁷ 
 -9.0×10⁶ 
 -8.0×10⁶ 
 -7.0×10⁶ 
 -6.0×10⁶ 
 -5.0×10⁶ 
 -4.0×10⁶ 
 -3.0×10⁶ 
 -2.0×10⁶ 
 -1.0×10⁶ 
 0 
 1.0×10⁶ 
 2.0×10⁶ 
 3.0×10⁶ 
 4.0×10⁶ 
 5.0×10⁶ 
 6.0×10⁶ 
 7.0×10⁶ 
 8.0×10⁶ 
 9.0×10⁶ 
 1.0×10⁷ 
 1.1×10⁷ 
 1.2×10⁷ 
 1.3×10⁷ 
 1.4×10⁷ 
 1.5×10⁷ 
 1.6×10⁷ 
 1.7×10⁷ 
 1.8×10⁷ 
 1.9×10⁷ 
 2.0×10⁷ 
 
 
 
 Erathostenes 
 Atkin 
 Sundaram 
 
 
 
 
 
 
 
 variable 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 -5.0 
 -4.8 
 -4.6 
 -4.4 
 -4.2 
 -4.0 
 -3.8 
 -3.6 
 -3.4 
 -3.2 
 -3.0 
 -2.8 
 -2.6 
 -2.4 
 -2.2 
 -2.0 
 -1.8 
 -1.6 
 -1.4 
 -1.2 
 -1.0 
 -0.8 
 -0.6 
 -0.4 
 -0.2 
 0.0 
 0.2 
 0.4 
 0.6 
 0.8 
 1.0 
 1.2 
 1.4 
 1.6 
 1.8 
 2.0 
 2.2 
 2.4 
 2.6 
 2.8 
 3.0 
 3.2 
 3.4 
 3.6 
 3.8 
 4.0 
 4.2 
 4.4 
 4.6 
 4.8 
 5.0 
 5.2 
 5.4 
 5.6 
 5.8 
 6.0 
 6.2 
 6.4 
 6.6 
 6.8 
 7.0 
 7.2 
 7.4 
 7.6 
 7.8 
 8.0 
 8.2 
 8.4 
 8.6 
 8.8 
 9.0 
 9.2 
 9.4 
 9.6 
 9.8 
 10.0 
 -5 
 0 
 5 
 10 
 -5.0 
 -4.5 
 -4.0 
 -3.5 
 -3.0 
 -2.5 
 -2.0 
 -1.5 
 -1.0 
 -0.5 
 0.0 
 0.5 
 1.0 
 1.5 
 2.0 
 2.5 
 3.0 
 3.5 
 4.0 
 4.5 
 5.0 
 5.5 
 6.0 
 6.5 
 7.0 
 7.5 
 8.0 
 8.5 
 9.0 
 9.5 
 10.0 
 
 
 elapsed time [s]

[type_doc]: http://julia.readthedocs.org/en/latest/manual/types/
[methods]: http://julia.readthedocs.org/en/latest/manual/methods/#man-methods
[eratosthenes]: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes
[sundaram]: https://en.wikipedia.org/wiki/Sieve_of_Sundaram